# Clasificación de Tumores Cerebrales con CNN (Entrenamiento y Validación 80/20)

Este notebook está preparado para el desarrollo y entrenamiento de una CNN sobre imágenes MRI. Incluye:
- Separación automática 80/20 para entrenamiento y validación
- Visualización de datos
- Definición y entrenamiento del modelo
- Guardado automático de checkpoints cada N épocas
- Tamaño de imagen: 512x512 píxeles

In [ ]:
# Importar bibliotecas necesarias
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    ModelCheckpoint,
    TensorBoard,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Configuración de semilla para reproducibilidad
np.random.seed(42)
tf.random.set_seed(42)

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Configuración de rutas
BASE_DIR = Path("../total/archive/Training")
MODEL_DIR = Path("models")
print(f"BASE_DIR: {BASE_DIR}")

# Detectar clases automáticamente
classes = [d.name for d in BASE_DIR.iterdir() if d.is_dir()]
print(f"Clases detectadas: {classes}")
num_classes = len(classes)

# Parámetros
IMG_SIZE = 512
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
L2_REG = 0.0001

In [ ]:
# Generador de datos con separación 80/20 (entrenamiento/validación)
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1],
    validation_split=0.2,
)

train_generator = train_datagen.flow_from_directory(
    str(BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    color_mode="rgb",
)

val_generator = train_datagen.flow_from_directory(
    str(BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
    color_mode="rgb",
)

Found 4604 images belonging to 4 classes.
Found 1148 images belonging to 4 classes.
Mapeo de clases: {'glioma': 0, 'meningioma': 1, 'no_tumor': 2, 'pituitary': 3}


In [ ]:
# Definir arquitectura de la CNN
def create_brain_tumor_model(
    input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=num_classes
):
    model = Sequential(
        [
            Conv2D(
                32,
                (3, 3),
                activation="relu",
                padding="same",
                input_shape=input_shape,
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            Conv2D(
                32,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(DROPOUT_RATE / 2),
            Conv2D(
                64,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            Conv2D(
                64,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(DROPOUT_RATE),
            Conv2D(
                128,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            Conv2D(
                128,
                (3, 3),
                activation="relu",
                padding="same",
                kernel_regularizer=l2(L2_REG),
            ),
            BatchNormalization(),
            MaxPooling2D((2, 2)),
            Dropout(DROPOUT_RATE),
            Flatten(),
            Dense(256, activation="relu", kernel_regularizer=l2(L2_REG)),
            BatchNormalization(),
            Dropout(DROPOUT_RATE),
            Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.AUC(name="auc")],
    )
    return model


c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 512, 512, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 262144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │   134,218,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,744,004 (514.01 MB)

 Trainable params: 134,741,508 (514.00 MB)

 Non-trainable params: 2,496 (9.75 KB)

In [ ]:
import glob
from tensorflow.keras.models import load_model

# Cargar el último modelo .keras guardado en MODEL_DIR o crear uno nuevo si no existe
model_files = sorted(glob.glob(str(MODEL_DIR / '*.keras')))
if model_files:
    last_model = model_files[-1]
    model = load_model(last_model)
else:
    model = create_brain_tumor_model()

model.summary()

In [ ]:
# Callbacks: EarlyStopping, ReduceLROnPlateau, y ModelCheckpoint cada N épocas
callbacks = [
    EarlyStopping(
        monitor="val_auc", patience=15, mode="max", restore_best_weights=True
    ),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=8),
    ModelCheckpoint(
        "best_model.keras", monitor="val_auc", save_best_only=True, mode="max"
    ),
    TensorBoard(log_dir="./logs"),
]

In [ ]:
# Entrenamiento del modelo con validación
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
)

c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5653 - loss: 1.2388
Epoch 1: saving model to c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\cnn\..\models\cnn_checkpoint_epoch_01.h5


143/143 ━━━━━━━━━━━━━━━━━━━━ 699s 5s/step - accuracy: 0.5658 - loss: 1.2373 - val_accuracy: 0.2464 - val_loss: 3.4368 - learning_rate: 0.0010
Epoch 2/200
  1/143 ━━━━━━━━━━━━━━━━━━━━ 10:31 4s/step - accuracy: 0.5000 - loss: 1.2917

c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\.venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: saving model to c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\cnn\..\models\cnn_checkpoint_epoch_02.h5


...

143/143 ━━━━━━━━━━━━━━━━━━━━ 599s 4s/step - accuracy: 0.8724 - loss: 0.3278 - val_accuracy: 0.5384 - val_loss: 2.6127 - learning_rate: 4.0000e-05
Epoch 28/200
  1/143 ━━━━━━━━━━━━━━━━━━━━ 9:09 4s/step - accuracy: 0.8438 - loss: 0.4932
Epoch 28: saving model to c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\cnn\..\models\cnn_checkpoint_epoch_28.h5


143/143 ━━━━━━━━━━━━━━━━━━━━ 52s 342ms/step - accuracy: 0.8438 - loss: 0.4932 - val_accuracy: 0.5393 - val_loss: 2.6695 - learning_rate: 4.0000e-05
Epoch 29/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8735 - loss: 0.3170
Epoch 29: saving model to c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\cnn\..\models\cnn_checkpoint_epoch_29.h5


143/143 ━━━━━━━━━━━━━━━━━━━━ 604s 4s/step - accuracy: 0.8735 - loss: 0.3169 - val_accuracy: 0.6598 - val_loss: 1.2541 - learning_rate: 4.0000e-05
Epoch 30/200
  1/143 ━━━━━━━━━━━━━━━━━━━━ 9:05 4s/step - accuracy: 0.7812 - loss: 0.3874
Epoch 30: saving model to c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\cnn\..\models\cnn_checkpoint_epoch_30.h5


143/143 ━━━━━━━━━━━━━━━━━━━━ 52s 339ms/step - accuracy: 0.7812 - loss: 0.3874 - val_accuracy: 0.6723 - val_loss: 1.2724 - learning_rate: 4.0000e-05
Epoch 31/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8791 - loss: 0.3158
Epoch 31: saving model to c:\Users\dev-max\Documents\GitHub\inteligencia_de_negocios\cnn\..\models\cnn_checkpoint_epoch_31.h5



Epoch 31: ReduceLROnPlateau reducing learning rate to 8.000000525498762e-06.
143/143 ━━━━━━━━━━━━━━━━━━━━ 596s 4s/step - accuracy: 0.8791 - loss: 0.3158 - val_accuracy: 0.7571 - val_loss: 0.8464 - learning_rate: 4.0000e-05
Epoch 31: early stopping
Restoring model weights from the end of the best epoch: 21.


In [ ]:
# Evaluar el modelo en el conjunto de validación
val_loss, val_acc, val_auc = model.evaluate(val_generator)
print(f"\nPrecisión Validación: {val_acc:.4f}")
print(f"AUC Validación: {val_auc:.4f}")

# Guardar el modelo final
final_model_path = MODEL_DIR / 'brain_tumor_cnn_model_final.h5'
model.save(str(final_model_path))
print(f"Modelo final guardado en: {final_model_path}")

In [ ]:
# Visualización de la curva de pérdida y precisión
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión del modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='lower right')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Pérdida del modelo')
plt.ylabel('Pérdida')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
plt.tight_layout()
plt.show()

**Notas:**
- El entrenamiento se realiza con separación 80/20 para entrenamiento y validación.
- Se guardan checkpoints automáticos cada N épocas y el modelo final al terminar.
- Puedes ajustar los hiperparámetros y arquitectura según tus necesidades.